In [2]:
import xml.etree.ElementTree as ET
import re
import MeCab
import unicodedata
import time


class Checker:
    def __init__(self, ng_words_file='assets/input/ngword_list.csv'):
        #self.xml_text = xml_text
        self.ng_words_file = ng_words_file
        self.mecab = MeCab.Tagger()

    def xml_checker(self, xml_text):
        try:
            root = ET.fromstring(xml_text)

            if root.tag != 'profile':
                return False

            expected_tags = {
                'hashtags', 'big_five_chart', 'character', 'personality',
                'hometown'
            }

            for child in root:
                if child.tag not in expected_tags:
                    return False
                if child.tag == 'big_five_chart':
                    expected_subtags = {'openness', 'conscientiousness',
                                        'extraversion', 'agreeableness', 'neuroticism'}
                    for subchild in child:
                        if subchild.tag not in expected_subtags:
                            return False

            return True
        except ET.ParseError:
            return False

    def xml_checker2(self, xml_text):
        try:
            root = ET.fromstring(xml_text)

            if root.tag != 'profile':
                return False

            expected_tags = {
                'occupation', 'favorite_things', 'hobby',
                'skill', 'habit', 'dream', 'talent', 'motto', 'comment'
            }

            # XML内のタグ集める
            found_tags = {child.tag for child in root}

            # 期待されるすべてのタグが存在するかチェック
            if not expected_tags.issubset(found_tags):
                return False

            for child in root:
                if len(child.text or '') > 30:
                    return False

            return True
        except ET.ParseError:
            return False


    def detect_ng_word(self, xml_text, pool=False):
        time2 = time.time()
        #半角を全角に
        xml_text_2 = unicodedata.normalize('NFKC', xml_text)
        # 記号系が、unicodeだと検出できない場合があるため、念のため両方。
        xml_text = xml_text + xml_text_2
        with open(self.ng_words_file, 'r', encoding='utf-8') as file:
            ng_words = [line.strip() for line in file]

        # 形態素解析でテキストを単語に分割
        node = self.mecab.parseToNode(xml_text)
        words = []
        while node:
            if node.surface:
                words.append(node.surface)
            node = node.next

        for i in range(len(words)-1):
            first_word = words[i]
            combined_two_word = words[i]+words[i+1]
            if i >= 1:
                combined_three_word = words[i-1]+combined_two_word
                if combined_three_word in ng_words:
                    print(f"NG word detected: {combined_three_word}")
                    return False
            
            if first_word in ng_words:
                print(f"NG word detected: {first_word}")
                return False
            
            if combined_two_word in ng_words:
                print(f"NG word detected: {combined_two_word}")
                return False
            

        return True
   

In [7]:
import os
import shutil
import time


def process_files(directory, checker, trash_dir, filtered_dir):
    # trash_output ディレクトリが存在しない場合は作成
    if not os.path.exists(trash_dir):
        os.makedirs(trash_dir)

    if not os.path.exists(filtered_dir):
        os.makedirs(filtered_dir)

    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(directory, filename)
            start_time = time.time()
            with open(file_path, 'r', encoding='utf-8') as file:
                xml_content = file.read()
                end_time = time.time()
                timeee = end_time - start_time
                print('time_load;', timeee)

            # NGワードのチェック
            if not checker.detect_ng_word(xml_content):

                new_filename = f"including_ng_word_{filename}"
                new_file_path = os.path.join(directory, new_filename)
                os.rename(file_path, new_file_path)
                print(f"Renamed '{filename}' to '{new_filename}'")

                # 新しいファイルをtrash_outputに移動
                trash_file_path = os.path.join(trash_dir, new_filename)

                shutil.move(new_file_path, trash_file_path)
                print(f"Moved '{new_filename}' to '{trash_dir}'")
            else:
                filtered_file_path = filtered_dir
                print(filtered_file_path)
                print(filtered_file_path)
                shutil.move(file_path, filtered_file_path)
                

In [13]:
checker = Checker('assets/input/ngword_list.csv')
# ディレクトリ内のファイルを処理
process_files('output-1.5-ver1.0_filtered/E_filtered_output', checker, 'output-1.5-ver1.0_filtered/E_trash_output','output-1.5-ver1.0_filtered/E_output/')

time_load; 0.037506103515625
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.012423992156982422
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.0015239715576171875
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.002359151840209961
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.0013229846954345703
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.0010368824005126953
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.012222051620483398
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.001512289047241211
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.007503986358642578
output-1.5-ver1.0_filtered/E_output/
output-1.5-ver1.0_filtered/E_output/
time_load; 0.0025610923767089844
output-1.5-ve

In [1]:
import os
import shutil

def rename_xml_files(base_dir, output_dir):
    print('ya')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print('make')

    count = 1
    sub_dirs = ['A_output', 'B_output', 'C_output', 'D_output','E_output']
    for sub_dir in sub_dirs:
        path = os.path.join(base_dir, sub_dir)
        for filename in sorted(os.listdir(path)):
            if filename.endswith('.xml'):
                new_filename = f'{count:05d}.xml'
                old_file_path = os.path.join(path, filename)
                new_file_path = os.path.join(output_dir, new_filename)
                shutil.copy(old_file_path, new_file_path)
                count += 1

# この関数を実行する前に、適切なパスを設定してください
rename_xml_files('/Users/yokoyamakai/gen-prof-ccjc/output/prompt_ver5.0', '/Users/yokoyamakai/gen-prof-ccjc/output/prompt_ver5.0/renamed_output')


ya
make


In [6]:
import os
import xml.etree.ElementTree as ET

def remove_factory_from_hometown(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(directory, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # <hometown> タグを見つけ、内容をチェックする
            for hometown in root.findall('hometown'):
                print(hometown.text)
                if '工場' in hometown.text:
                    # '工場' を空白に置換する
                    hometown.text = hometown.text.replace('工場', '')
                    print(hometown.text)

            # XMLファイルを上書き保存する
            tree.write(file_path, encoding='utf-8')

# この関数を実行する前に、適切なパスを設定してください
remove_factory_from_hometown("/Users/yokoyamakai/Desktop/output-1.5-ver1.0_filtered_renamed/output-ver1.2-filtered_renamed")


熊本
多摩
花巻
熊本
えびの
明石
鳥栖
札幌
広島
明石
熊本
明石
えびの
熊本
岩槻
広島
多摩
明石
明石
埼玉
蔵王
札幌
埼玉
岩槻
基山
多摩
えびの
京都
札幌
花巻
熊本
えびの
基山
埼玉
岩槻
京都
基山
熊本
花巻
明石
明石
熊本
花巻
埼玉
蔵王
えびの
えびの
明石
基山
京都
熊本
広島
基山
蔵王
埼玉
埼玉
花巻
埼玉
京都
熊本
埼玉
熊本
花巻
蔵王
広島
京都
広島
鳥栖
広島
札幌
京都
広島
多摩
京都
基山
岩槻
花巻
多摩
岩槻
埼玉
花巻
鳥栖
岩槻
明石
蔵王
基山
鳥栖
京都
花巻
基山
熊本
広島
鳥栖
岩槻
熊本
埼玉
花巻
えびの
京都
熊本
広島
蔵王
基山
えびの
札幌
京都
岩槻
蔵王
鳥栖
広島
明石
広島
明石
多摩
えびの
えびの
多摩
明石
明石
札幌
広島
広島
基山
えびの
埼玉
明石
熊本
明石
花巻
花巻
基山
熊本
花巻
花巻
札幌
花巻
えびの
えびの
基山
明石
鳥栖
広島
蔵王
埼玉
熊本
岩槻
多摩
えびの
多摩
広島
熊本
広島
札幌
京都
京都
京都
明石
鳥栖
蔵王
えびの
熊本
岩槻
えびの
基山
岩槻
明石
埼玉
蔵王
明石
京都
基山
広島
札幌
蔵王
多摩
多摩
埼玉
明石
埼玉
えびの
札幌
京都
基山
明石
多摩
鳥栖
明石
熊本
札幌
岩槻
明石
埼玉
蔵王
岩槻
蔵王
札幌
多摩
京都
多摩
多摩
基山
岩槻
多摩
多摩
埼玉
鳥栖
花巻
基山
埼玉
えびの
熊本
京都
岩槻
埼玉
広島
多摩
岩槻
京都
京都
蔵王
基山
蔵王
熊本
明石
えびの
京都
えびの
岩槻
蔵王
埼玉
鳥栖
明石
基山
基山
花巻
埼玉
京都
札幌
花巻
基山
鳥栖
蔵王
えびの
札幌
岩槻
埼玉
鳥栖
岩槻
広島
埼玉
明石
札幌
花巻
札幌
花巻
岩槻
京都
広島
えびの
明石
花巻
基山
京都
鳥栖
鳥栖
京都
京都
花巻
基山
埼玉
明石
蔵王
えびの
明石
岩槻
多摩
基山
基山
札幌
岩槻
京都
札幌
札幌
広島
鳥栖
基山
熊本
岩槻
札幌
蔵王
花巻
鳥栖
京都
蔵王
明石
広島
京都
京都
熊本
花巻
埼玉
広島
埼玉
埼玉
札幌
多摩
熊本
花巻
広島
花巻
広島
広島
えびの
えびの
基山
えびの
鳥栖
花巻
蔵王
埼玉
基山
広島
明石
熊本
蔵王

置換する作業

In [14]:
xml_dic = '/Users/yokoyamakai/240131_ccjcoutput/goodrating'

In [15]:
import os
import xml.etree.ElementTree as ET

def replace_words_in_xml(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(directory, filename)
            tree = ET.parse(file_path)
            root = tree.getroot()

            replaced = False

            # すべてのテキストノードを探索して、必要な置換を行う
            for elem in root.iter():
                if elem.text is not None and ('地下アイドル' in elem.text or 'オタク' in elem.text):
                    elem.text = elem.text.replace('地下アイドル', 'アイドル').replace('オタク', 'ファン')
                    replaced = True

            # 置換が行われた場合、XMLの内容を出力
            if replaced:
                print("置換しました")
                ET.dump(root)

                # XMLファイルを上書き保存する
                tree.write(file_path, encoding='utf-8')
# この関数を実行する前に、適切なパスを設定してください
replace_words_in_xml(xml_dic)


置換しました
<profile>
<hashtags>#ラーメンめぐり #ツーリズム #食べ歩き #旅行 #グルメ #ファン #食べすぎ #ラーメン職人 #一杯 #スープ #麺 #チャーシュー</hashtags>
<big_five_chart>
<openness>30</openness>
<conscientiousness>60</conscientiousness>
<extraversion>50</extraversion>
<agreeableness>40</agreeableness>
<neuroticism>70</neuroticism>
</big_five_chart>
<character>努力家</character>
<personality>開放的な性格を持ちつつも、勤勉で真面目な一面があります。外向的ではなく内向的ながら、他人に対しても協調的であるため、交流が得意です。感情の起伏が大きく、神経質な一面がありますが、それが彼の創作活動により緻密さをもたらします。ラーメンへの情熱は人一倍で、職人としての腕前も非常に高いです。京都出身で、着物が似合う端正な容姿を持っています。話し方は穏やかで、丁寧な口調で話します。</personality>
<hometown>京都</hometown>
<occupation>ラーメン試食家</occupation>
<favorite_things>寺院</favorite_things>
<hobby>日本史の研究</hobby>
<skill>ラーメンを一口で分析する</skill>
<habit>毎晩星を眺める</habit>
<dream>世界中のラーメンを食べ歩く</dream>
<talent>短歌を詠むこと</talent>
<motto>「一杯一杯に心を込めて」</motto>
<comment>おぬしはどんな味が好なんじゃ？</comment>
</profile>
置換しました
<profile>
<hashtags>#アイドル #ステージ #熱狂 #ファンミーティング #楽曲制作 #ライブ #ダンス #声優 #トレーニング #アイドルコンテスト #みんなと一緒に #夢</hashtags>
<big_five_chart>
<openness>40</openness>

In [ ]:
/Users/yokoyamakai/Desktop/ccjcBUver1/00001_4.xml